In [1]:
%run stdPackages.ipynb
os.chdir(d['py'])
import US_EulerModel_main, US_EulerModel_c, US_EulerModel_policy
os.chdir(d['curr'])

# Social Security Design for US (Euler, finite horizon): 
## Identification of structural parameters $(\xi, \theta)$ for 2010 model

This notebook considers the model for the US where all household types are "Euler types" (e.g. no hand-to-mouth or unemployed). We search the parameter space of the coefficient of relative risk aversion ($\theta$) and frisch elasticity $(\xi)$ of households to identify combinations that ensures the model replicates the choice of pension characteristics of 2010.

The model uses a finite horizon version to solve for the CRRA model. We revisit the robustness of results in other notebooks.

## 1. Data and settings

Data on inputs:

In [2]:
t0date = 1980
%run USEuler_Data.ipynb

Initial parameter values and settings:

In [3]:
ω0 = 1.5
ngrid = 50

Grid specification for searches over $\rho, \xi$:

In [4]:
ξ0, ξLower, nξ = .5, .1, 9
ρ0, ρLower, nρ = 2.5, 1, 11

Initialize model:

In [5]:
m = US_EulerModel_main.Model(ngrid = ngrid, ξ = ξ0, ρ = ρ0, ω = ω0, **data)

Specify others settings: Size of grids to use in grid search methods and the style of solution to use (see ```USEuler_Policy.ipynb``` on the different methods):

In [6]:
m.ESC.grids['τ'] = m.ESC.nGrids('τ', 101) # create standard grid with 101 steps for grid searches for τ
m.ESC.grids['κ'] = m.ESC.nGrids('κ', 51) # create standard grid with 51 steps for grid searches for κ
m.ESC.kwargs_t = {'style': 'VeryRobust'} # use the "VeryRobust" style of solving the model.
m.ESC.kwargs_T = {'style': 'VeryRobust'} # use the "VeryRobust" style of solving the model in terminal state.

## 2. Calibration

We calibrate the model on a grid of $\rho$ values. For each value, it is somewhat straightforward to target policy rates ($\tau_t$) and interest rates $(R_t)$, but identifying the correct $\xi$ that ensures a $\kappa$ target is more tricky. This step is done using an initial grid search (with calibrations of $\tau_t, R_t$ in each step) and then a subsequent golden-section-like search. Once this is completed, we update $\rho$ and start over. 

*Grids:*

In [7]:
β0 = m.US_βinv()
ρgrid = np.round(np.linspace(ρ0, ρLower, nρ), 2)
ξgrid = np.round(np.linspace(ξ0, ξLower, nξ), 3)

*Start from pickle instead:*

In [8]:
with open(os.path.join(d['data'], 'USEuler1980_gridsearch'), "rb") as file:
    solsGrid = pickle.load(file)

To be able to "start" the model from a reasonable parameter range, let us start by solving on a grid of $\xi$:

In [9]:
# sols0 = m.USCal_OnGrid(ξgrid, data['t0'], var = 'ξ')

Then, given this, solve on a grid of $\xi$ for each step. Create a "break" variable as follows: If there has been no solution with a sign change on the grid of $\xi$ yet, keep iterating. The first time a sign change is recorded update local to ```solved```. Then, whenever there is no longer a sign change, stop iteration:

In [10]:
# solved = False
# solsGrid = dict.fromkeys(ξgrid)
# for i in range(len(ξgrid)):
#     m.db['ξ'] = ξgrid[i]
#     m.US_Xi()
#     m.db['ω'] = sols0['ω'][i]
#     m.db['βi'] = m.US_β(sols0['β'][i])
#     solsGrid[m.db['ξ']] = {'ρ' : ρgrid, 'obj': np.empty(ρgrid.size)}
#     solsGrid[m.db['ξ']].update({k: solsGrid[m.db['ξ']]['obj'].copy() for k in ('ω','β')})
#     for j in range(ρgrid.size):
#         solsGrid[m.db['ξ']] = m.USCal_OnGrid_i(data['t0'], solsGrid[m.db['ξ']], j, var = 'ρ')
#     # solsGrid[ξgrid[i]] = m.USCal_OnGrid(ρgrid, data['t0'], var = 'ρ')
#     if solved is False:
#         if np.nonzero(np.diff(np.sign(solsGrid[ξgrid[i]]['obj'])) != 0)[0].size>0:
#             solved = True
#     else:
#         if np.nonzero(np.diff(np.sign(solsGrid[ξgrid[i]]['obj'])) != 0)[0].size == 0:
#             break

Store data:

In [11]:
# with open(os.path.join(d['data'], 'USEuler1980_gridsearch'), "wb") as file:
#     pickle.dump(solsGrid, file)

### 2.1. Search for solutions on relevant part of grid.

Remove ```None``` entries and only keep sign changes:

In [12]:
solsSignChange = {k: v for k,v in solsGrid.items() if v}
solsSignChange = {k: v for k,v in solsSignChange.items() if np.nonzero(np.diff(np.sign(v['obj'])) != 0)[0].size>0}

Solve on this grid:

In [13]:
%%time
cals = dict.fromkeys(solsSignChange)
for k,v in solsSignChange.items():
    m.db['ξ'] = k
    m.US_Xi() # update Xi based on ξ
    sc = m.USCal_SCidx(v) # idx for sign change
    m.db['ω'] = v['ω'][sc]
    m.db['βi'] = m.US_β(v['β'][sc])
    cals[k] = m.USCal_GoldenSection(v['ρ'][sc:sc+2], data['t0'], n = 2, tol = 1e-5, iterMax = 5, var = 'ρ')
    print(k)

0.5
0.45
0.4
0.35
0.3
0.25
0.2
0.15
0.1
CPU times: total: 8min 53s
Wall time: 8min 19s


Store solution dictionary and model instance:

In [14]:
with open(os.path.join(d['data'], 'USEuler1980_cals'), "wb") as file:
    pickle.dump(cals, file)
with open(os.path.join(d['data'], 'mUSEuler1980'), "wb") as file:
    pickle.dump(m, file)